<a href="https://colab.research.google.com/github/enginearn/llm_book_for_intro/blob/main/llm_book_for_intro_ch_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 1

In [ ]:
!pip install transformers[ja,sentecepiece,touch]

In [4]:
from transformers import pipeline

## Text classification

In [10]:
from transformers.tools import text_classification
text_classification_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-marc_ja"
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--llm-book--bert-base-japanese-v3-marc_ja/snapshots/7b47edf80477fd9da0ee1bc1908326ac012d624f/config.json
Model config BertConfig {
  "_name_or_path": "llm-book/bert-base-japanese-v3-marc_ja",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.31.0",
  "type_vocab_size"

In [11]:
positive_text = "明日は明日の風が吹く"
print(text_classification_pipeline(positive_text)[0])

{'label': 'positive', 'score': 0.998440682888031}


In [12]:
negative_text = "お金がなさ過ぎてつらい..."
print(text_classification_pipeline(negative_text)[0])

{'label': 'negative', 'score': 0.9948321580886841}


## Natural Language Inference: 自然言語推論

In [13]:
nli_pipline = pipeline(model="llm-book/bert-base-japanese-v3-jnli")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--llm-book--bert-base-japanese-v3-jnli/snapshots/9056fce079ed3fc284c9b2d1c2abccae3d13af61/config.json
Model config BertConfig {
  "_name_or_path": "llm-book/bert-base-japanese-v3-jnli",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "contradiction",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 1,
    "entailment": 0,
    "neutral": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transf

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--llm-book--bert-base-japanese-v3-jnli/snapshots/9056fce079ed3fc284c9b2d1c2abccae3d13af61/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at llm-book/bert-base-japanese-v3-jnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--llm-book--bert-base-japanese-v3-jnli/snapshots/9056fce079ed3fc284c9b2d1c2abccae3d13af61/vocab.txt
loading file spiece.model from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--llm-book--bert-base-japanese-v3-jnli/snapshots/9056fce079ed3fc284c9b2d1c2abccae3d13af61/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--llm-book--bert-base-japanese-v3-jnli/snapshots/9056fce079ed3fc284c9b2d1c2abccae3d13af61/tokenizer_config.json


`entailment`は`含意`であり、"二人の女性が山を眺めています"が成立するならば、"山を眺めている人が2人います"も成立するという関係を表す。

In [14]:
text = "二人の女性が山を眺めています"
entailment_text = "山を眺めている人が2人います"

print(nli_pipline({"text": text, "text_pair": entailment_text}))

{'label': 'entailment', 'score': 0.9933545589447021}


`contradiction`は、`矛盾`

In [21]:
contradiction_text = "女性２人が山を破壊しています"

print(nli_pipline({"text": text, "text_pair": contradiction_text}))

{'label': 'contradiction', 'score': 0.9771778583526611}


In [22]:
neutral_text = "女性２人が山で料理をしています"

print(nli_pipline({"text": text, "text_pair": neutral_text}))

{'label': 'neutral', 'score': 0.9970123767852783}


## Semantic textual similarity: STS

二つのテキストが似ている度合いをスコアとして予測するタスク